In [ ]:
import json
import duckdb
import boto3
from pathlib import Path
import logging
import shutil

ddb_temp_directory = "/tmp/duckdb"
# Set up structured logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def create_ddb_home_directory() -> None:
    """
    Function to create a new home directory.
    If exists, delete all files and create new directory
    Args:
        path (str): Path to the directory to delete.
    """
    path = ddb_temp_directory
    directory_path = Path(path)
    if directory_path.exists():
        shutil.rmtree(path)
        logging.info(f"Deleting folder {path}")
    
    _=directory_path.mkdir(parents=True,exist_ok=True)
    
    return None

def lambda_handler(event, context):
    """
    AWS Lambda function to execute a DuckDB query using temporary storage.
    
    Args:
        event (dict): Event data containing query parameters.
        context (object): Lambda runtime information.
    
    Returns:
        dict: Status and result of query execution.
    """
    # For structured logging
    request_id = context.aws_request_id
    _=create_ddb_home_directory()
    
    # Retrieve AWS credentials
    session = boto3.Session()


    # Initialize DuckDB connection
    con = duckdb.connect(":memory:")
    setup_query = f"""
        SET home_directory='{ddb_temp_directory}';
        PRAGMA temp_directory='{ddb_temp_directory}'; 
        INSTALL httpfs; LOAD httpfs;
        INSTALL spatial; LOAD spatial;
        INSTALL AWS; LOAD AWS;
        CREATE SECRET secret2 (TYPE S3, PROVIDER CREDENTIAL_CHAIN);
    """
    _=con.sql(setup_query)

    # Ensure query exists in the event
    if "qry" not in event:
        error_message = "Missing 'qry' parameter in the event."
        logger.error(json.dumps({"request_id": request_id, "status": "error", "message": error_message}))
        return {
            "StatusCode": 400, # bad request error code
            "result": "error",
            "message": error_message
        }

    query = event["qry"]
    if "qry_info" in event:
        logger.info(f"Query Info: {event['qry_info']}")

    # Execute the query
    try:
        logger.info(json.dumps({"request_id": request_id, "status": "started", "query": query}))
        result = con.sql(query)
        result = result.fetchall() if result else "None"
        
        # Log success
        logger.info(json.dumps({"request_id": request_id, "status": "success"}))

        return {
            "StatusCode": 200,
            "result": result
        }

    except Exception as e:
        # Log error
        error_message = str(e)
        logger.error(json.dumps({"request_id": request_id, "status": "error", "message": error_message}))
        
        return {
            "StatusCode": 400,
            "result": "error",
            "message": error_message,
        }